In [1]:
from kafka import KafkaConsumer
from bs4 import BeautifulSoup
import json
import requests

In [2]:
def get(url):
    res = requests.get(url)
    if res.status_code == 200:
        return res.text.strip()
    else:
        return ''

In [5]:
def parse(html):
    soup = BeautifulSoup(html, 'html.parser')
    
    title_section = soup.select('.recipe-summary__h1')
    calories_section = soup.select('.calorie-count')
    submitter_section = soup.select('.submitter__name')
    description_section = soup.select('.submitter__description')
    ingredients_section = soup.select('.recipe-ingred_txt')
    
    if calories_section:
            calories = calories_section[0].text.strip().replace(' cals', '')

    ingreds = []
    if ingredients_section:
        for ingredient in ingredients_section:
            ingredient_text = ingredient.text.strip()
            if 'Add all ingredients to list' not in ingredient_text and ingredient_text != '':
                ingreds.append({'step': ingredient.text.strip()})

    if description_section:
        description = description_section[0].text.strip().replace('"', '')

    if submitter_section:
        submit_by = submitter_section[0].text.strip()

    if title_section:
        title = title_section[0].text.strip()

    if title != "-":
        rec = {'title': title, 'submitter': submit_by, 'description': description, 'calories': calories,
                   'ingredients': ingreds}
    else:
        rec = {}
    
    return json.dumps(rec, indent=4)

In [6]:
consumer = KafkaConsumer('recipe',
                         auto_offset_reset='earliest',
                         bootstrap_servers=['localhost:9092'],
                         api_version=(0, 10),
                         consumer_timeout_ms=1000)

for msg in consumer:
    url = msg.value
    result = parse(get(url))
    if result != "{}":
        print (result)